In [24]:
import tensorflow as tf

In [7]:
import tensorflow_hub as hub

In [8]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [9]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

In [95]:
cap=cv2.VideoCapture('running.mp4')
while cap.isOpened():
    ret,frame=cap.read()
    #Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img,axis=0),128,256)
    input_img = tf.cast(img,dtype=tf.int32)
    
    #Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # Render keypoints 
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.2)
    
    cv2.imshow('Movenet multipose',frame)
    
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [83]:
frame.shape

(676, 1280, 3)

In [84]:
676/1280

0.528125

In [85]:
256*.52

133.12

In [86]:
133%32

5

In [87]:
133-5

128

In [88]:
256*.52

133.12

In [89]:
keypoints_with_scores[0]

array([[0.14957061, 0.560165  , 0.3781239 ],
       [0.13688801, 0.5660148 , 0.5743091 ],
       [0.13473761, 0.5521133 , 0.5163709 ],
       [0.14913991, 0.5739149 , 0.49236777],
       [0.14553934, 0.5403045 , 0.60453874],
       [0.23029318, 0.59569526, 0.6819643 ],
       [0.23181199, 0.5142273 , 0.6649099 ],
       [0.35400432, 0.619764  , 0.7793927 ],
       [0.34166238, 0.49533615, 0.48890907],
       [0.3548336 , 0.58099157, 0.5064776 ],
       [0.36903492, 0.51325107, 0.45426142],
       [0.4587857 , 0.5768876 , 0.6693453 ],
       [0.46397159, 0.5342734 , 0.63104784],
       [0.6331107 , 0.57295936, 0.6648483 ],
       [0.6437543 , 0.53277194, 0.6535472 ],
       [0.76400226, 0.56988966, 0.6197769 ],
       [0.8244408 , 0.5293817 , 0.5096425 ]], dtype=float32)

In [29]:
# x,y,score
3*17

51

In [31]:
results['output_0'].numpy()[:,:,:51].shape

(1, 6, 51)

In [38]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [93]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 3, (0,255,0), -1)

In [92]:
EDGES = {
    
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [91]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)